# Retrieve EU Gesetzt

Code explanation in the file 'eu_gesetzt_code_explanation.ipynb'

### Ongoing notes:

Each law has a total different html structure, so I could not run one single loop to all of them. 
I'll try to roll te loop with pdf, because the final structure is similar. <br>
The first law (Daten Governance Act) is ready  :)


In [3]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import unicodedata

In [4]:
## Getting the law: Daten-Governance-Rechtsakt

URL = "https://eur-lex.europa.eu/legal-content/DE/TXT/HTML/?uri=CELEX:32022R0868&qid=1666873096926&from=EN", 
law = "Daten-Governance-Rechtsakt"
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib') 


## Getting all the artikels and separating Artikels with and without Absatze:

#1. getting all artikels and absatze:

ids = [tag['id'] for tag in soup.select('div[id]')]

df = pd.DataFrame(ids)
df.rename(columns={list(df)[0]:'ids'}, inplace=True)
df['artikel'] = df['ids'].str[:3]
df

#2. Looking which one of them have Absatze

df1 = df.groupby(['artikel']).count().reset_index()
df1 = df1.query("ids > 1")
ids_mit = df1['artikel']#.astype('str')

#3. Getting the final ids, so the ids and ids with absatze dont appear twice:
ids_final = set(ids) - set(ids_mit)

## Creating the table with all Artikel and Absatze:

#1. Creating empty df:
df_law = pd.DataFrame(columns = ['gesetzt', 'artikel', 'absatz', 'text'])

#2. loop for ids:

for id_entry in ids_final:
    #pegando tudo:
    table = soup.find('div', attrs = {'id':id_entry}) 
    my_str = table.text
    result = unicodedata.normalize('NFKD', my_str)   # removind \xa0
    result = " ".join(result.split())
    #result # lindo, está o absatz completo
    #agora pegando os artigos e os absatz
    #criando um df temporario com os detalhes do artigo:
    if len(id_entry) < 4:
        df_temp = pd.DataFrame({'gesetzt': [law],
                                'artikel': [id_entry.split('.')[0]], 
                                'absatz' : ["NA"],
                                'text': [result]})
    else:
        df_temp = pd.DataFrame({'gesetzt': [law],
                                'artikel': [id_entry.split('.')[0]], 
                                'absatz' : [id_entry.split('.')[1]],
                                'text': [result]})
    #df_temp.dtypes funciona
    df_law = pd.concat([df_law, df_temp])

df_law

df_law.to_csv("df_eu_law.csv")

c:\Python310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
